In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from smooth.adam_general.core.adam import ADAM
import numpy as np
import pandas as pd


# Simulation-based Prediction Intervals Tests - Python Implementation

Testing simulation-based prediction intervals with different model types. Results should match R implementation when using same seeds and data.


### Test 1: Additive ETS(A,N,N) - simulation vs parametric


In [3]:
np.random.seed(42)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [4]:
model = ADAM(model='ANN', lags=[1], initial='optimal')
model.fit(ts_df)
print('ETS(A,N,N) fitted model:')
print('Alpha:', model.adam_estimated['B'][0])
print('Initial level:', model.adam_estimated['B'][1])
print()

# Parametric intervals
result_param = model.predict_intervals(h=12, interval_method='parametric', random_state=42, nsim=1000)
print('Parametric intervals (first 3 steps):')
print('Mean:', result_param['forecast'][:3])
print('Lower:', result_param['lower'][:3])
print('Upper:', result_param['upper'][:3])



=== _extract_initial_states DEBUG ===
ets_model: True
model_is_trendy: False
model_is_seasonal: False
s_type: N
lags_model_seasonal: []
mat_vt shape: (1, 101)
mat_vt[:, 0] (first column): [98.28701439]
Extracted initial_states keys: ['level']
  level: 98.28701438558171


=== _extract_initial_states DEBUG ===
ets_model: True
model_is_trendy: False
model_is_seasonal: False
s_type: N
lags_model_seasonal: []
mat_vt shape: (1, 101)
mat_vt[:, 0] (first column): [98.28701439]
Extracted initial_states keys: ['level']
  level: 98.28701438558171

ETS(A,N,N) fitted model:
Alpha: 0.0
Initial level: 98.96078869335756



ValueError: Of the four parameters: start, end, periods, and freq, exactly three must be specified

In [ ]:
# Simulation intervals with fixed seed
result_sim = model.predict_intervals(h=12, interval_method='simulation', random_state=42, nsim=1000)
print('Simulation intervals (first 3 steps, nsim=1000):')
print('Mean:', result_sim['forecast'][:3])
print('Lower:', result_sim['lower'][:3])
print('Upper:', result_sim['upper'][:3])


### Test 2: Multiplicative seasonal ETS(A,N,M) - should trigger simulation


In [ ]:
np.random.seed(123)
n_points = 60
seasonal_pattern = np.tile([0.8, 1.0, 1.2, 1.1], n_points // 4)
time_series_seasonal = 100 + np.random.normal(0, 5, n_points) + seasonal_pattern * 10
ts_seasonal = pd.DataFrame({'value': time_series_seasonal}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [ ]:
model_anm = ADAM(model='ANM', lags=[1, 4], initial='optimal')
model_anm.fit(ts_seasonal)
print('ETS(A,N,M) fitted model:')
print('Alpha:', model_anm.adam_estimated['B'][0])
print('Gamma:', model_anm.adam_estimated['B'][1])
print()

# "prediction" should map to simulated for multiplicative seasonality
result_pred = model_anm.predict_intervals(h=8, interval_method='prediction', random_state=123, nsim=1000)
print('Prediction intervals (should use simulation for M seasonality):')
print('Mean:', result_pred['forecast'][:4])
print('Lower:', result_pred['lower'][:4])
print('Upper:', result_pred['upper'][:4])


### Test 3: Small nsim normalization check (nsim=100)


In [ ]:
# Test with small nsim to ensure normalization is applied
model_test = ADAM(model='ANN', lags=[1], initial='optimal')
model_test.fit(ts_df)

result_small = model_test.predict_intervals(h=6, interval_method='simulation', random_state=789, nsim=100)
print('Simulation with nsim=100 (normalization should apply):')
print('Mean:', result_small['forecast'][:3])
print('Lower:', result_small['lower'][:3])
print('Upper:', result_small['upper'][:3])


### Test 4: Test scenarios return


In [ ]:
# Test scenarios return
result_scenarios = model.predict_intervals(h=6, interval_method='simulation', random_state=42, nsim=500, scenarios=True)
print('With scenarios=True:')
print('Forecast shape:', result_scenarios['forecast'].shape)
if 'scenarios' in result_scenarios:
    print('Scenarios shape:', result_scenarios['scenarios'].shape)
    print('Scenarios should be (h, nsim) = (6, 500)')
else:
    print('Scenarios not returned (check DataFrame attrs)')
